In [1]:
%use fuel(2.3.1)

In [2]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [3]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 22

In [4]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get().trim()
}

In [5]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
val sample = """on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10"""

In [7]:
val input = getInput(year, day, session)

In [8]:
class KDNode(val range: List<Pair<Int, Int>>, var state: Boolean) {
    var children: List<KDNode> = listOf()
    
    fun containedBy(range: List<Pair<Int, Int>>): Boolean {
        return (0 until range.size).all {
            this.range[it].first >= range[it].first && this.range[it].second <= range[it].second
        }
    }
    
    fun outsideOf(range: List<Pair<Int, Int>>): Boolean {
        return (0 until range.size).any {
            this.range[it].first >= range[it].second || this.range[it].second <= range[it].first
        }
    }
    
    fun setState(range: List<Pair<Int, Int>>, state: Boolean) {
        if (this.outsideOf(range))
            return
        
        if (this.containedBy(range)) {
            this.state = state
            this.children = listOf()
            return
        }
        
        if (this.children.isEmpty()) {
            val sub = (0 until range.size).map {
                setOf(range[it].first, range[it].second, this.range[it].first, this.range[it].second)
                    .filter { i -> i >= this.range[it].first && i <= this.range[it].second }
                    .toList()
                    .sorted()
                    .windowed(2)
                    .map { Pair(it[0], it[1]) }
                    .toSet()
            }
                    
            this.children = sub.fold(listOf(listOf<Any?>())) { acc, set ->
                acc.flatMap { list -> set.map { element -> list + element } }
            }.map {
                KDNode(it as List<Pair<Int, Int>>, this.state)
            }
        }
        
        for (child in this.children)
            child.setState(range, state)
    }
    
    fun calc(): Long {
        if (this.children.isEmpty()) {
            if (this.state) {
                var ans = 1L
                for (i in 0 until this.range.size)
                    ans *= (this.range[i].second - this.range[i].first)
                return ans
            } else {
                return 0L
            }
        } else {
            return this.children.map { it.calc() }.sum()
        }
    }
}

In [9]:
fun solve(input: String, k: Int, inf: Int): String {
    val root = KDNode(buildList { repeat(k) { add(Pair(-inf, inf + 1)) }}, false)
        
    for (line in input.split("\n")) {
        val state = line[1] == 'n'
        val pos = Regex("""-?\d+""").findAll(line).map { it.groupValues[0].toInt() }.toList()
        root.setState(listOf(Pair(pos[0], pos[1] + 1), Pair(pos[2], pos[3] + 1), Pair(pos[4], pos[5] + 1)), state)
    }
    
    return root.calc().toString()
}

In [10]:
fun partOne(input: String): String {
    return solve(input, 3, 100)
}

In [11]:
partOne(sample)

39

In [12]:
val partOneAns = partOne(input)
partOneAns

648681

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [13]:
fun partTwo(input: String): String {
    return solve(input, 3, 1000000)
}

In [14]:
partTwo(sample)

39

In [15]:
val partTwoAns = partTwo(input)
partTwoAns

1302784472088899

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)